In [1]:
import requests
import json

In [2]:
import logging
logging.basicConfig(level=logging.INFO)

In [3]:
# Generic Function to send GET requests to DraCor API

#corpusname:str -> []
def get(**kwargs):
    #corpusname=corpusname
    #playname=playname
    #apibase="https://dracor.org/api/"
    #method=method
    #parse_json: True
    
    #could set different apibase, e.g. https://staging.dracor.org/api/ [not recommended, pls use the production server]
    if "apibase" in kwargs:
        if kwargs["apibase"].endswith("/"):
            apibase = kwargs["apibase"]
        else:
            apibase = kwargs["apibase"] + "/"
    else:
        #use local API per default
        apibase = "http://dracor.org/api/"
    if "corpusname" in kwargs and "playname" in kwargs:
        # used for /api/corpora/{corpusname}/play/{playname}/
        if "method" in kwargs:
            request_url = apibase + "corpora/" + kwargs["corpusname"] + "/play/" + kwargs["playname"] + "/" + kwargs["method"]
        else:
            request_url = apibase + "corpora/" + kwargs["corpusname"] + "/play/" + kwargs["playname"]
    elif "corpusname" in kwargs and not "playname" in kwargs:
        if "method" in kwargs:
            request_url = apibase + "corpora/" + kwargs["corpusname"] + "/" + kwargs["method"]
        else:
            request_url = apibase + "corpora/" + kwargs["corpusname"] 
    elif "method" in kwargs and not "corpusname" in kwargs and not "playname" in kwargs:
            request_url = apibase + kwargs["method"]
            
    else: 
        #nothing set
        request = request_url = apibase + "info"
    
    #send the response
    r = requests.get(request_url)
    if r.status_code == 200:
        #success!
        if "parse_json" in kwargs:
            if kwargs["parse_json"] == True:
                json_data = json.loads(r.text)
                return json_data
            else:
                return r.text
        else:
            return r.text
    else:
        raise Exception("Request was not successful. Server returned status code: "  + str(r.status_code))

In [4]:
from dracor_api_response_schemas import is_valid

## /info

In [5]:
from dracor_api_response_schemas import InfoSchema

In [6]:
info_response = get(method="info",parse_json=True)

In [7]:
info_response

{'name': 'DraCor API',
 'version': '0.89.0',
 'status': 'beta',
 'existdb': '6.0.1'}

In [8]:
# Validate against the schema
# In case of an validation Error (returned value is False), set the flag "catch_error=False", which will return the 
# message from the exception
is_valid(info_response, InfoSchema)

INFO:root:Validation passed.


True

## /corpora

In [9]:
from dracor_api_response_schemas import CorpusInCorporaSchema

In [10]:
corpora_response_no_metrics = get(method="corpora", parse_json=True)

In [11]:
print("Validation of /corpora without metrics included:")
for corpus in corpora_response_no_metrics:
    if is_valid(corpus, CorpusInCorporaSchema):
        print("Data of corpus " + corpus["title"] + " passed validation.")
    else:
        print ("ERROR " + corpus["title"])

INFO:root:Validation passed.
INFO:root:Validation passed.
INFO:root:Validation passed.
INFO:root:Validation passed.
INFO:root:Validation passed.
INFO:root:Validation passed.
INFO:root:Validation passed.
INFO:root:Validation passed.
INFO:root:Validation passed.
INFO:root:Validation passed.
INFO:root:Validation passed.
INFO:root:Validation passed.
INFO:root:Validation passed.
INFO:root:Validation passed.
INFO:root:Validation passed.


Validation of /corpora without metrics included:
Data of corpus Alsatian Drama Corpus passed validation.
Data of corpus Bashkir Drama Corpus passed validation.
Data of corpus Calderón Drama Corpus passed validation.
Data of corpus French Drama Corpus passed validation.
Data of corpus German Drama Corpus passed validation.
Data of corpus German Shakespeare Drama Corpus passed validation.
Data of corpus Greek Drama Corpus passed validation.
Data of corpus Hungarian Drama Corpus passed validation.
Data of corpus Italian Drama Corpus passed validation.
Data of corpus Roman Drama Corpus passed validation.
Data of corpus Russian Drama Corpus passed validation.
Data of corpus Shakespeare Drama Corpus passed validation.
Data of corpus Spanish Drama Corpus passed validation.
Data of corpus Swedish Drama Corpus passed validation.
Data of corpus Tatar Drama Corpus passed validation.


In [12]:
corpora_response_incl_metrics = get(method="corpora?include=metrics", parse_json=True)

In [13]:
print("Validation of /corpora with metrics included:")
for corpus in corpora_response_incl_metrics:
    if is_valid(corpus, CorpusInCorporaSchema):
        print("Data of corpus " + corpus["title"] + " passed validation.")
    else:
        print ("ERROR " + corpus["title"])

INFO:root:Validation passed.
INFO:root:Validation passed.
INFO:root:Validation passed.
INFO:root:Validation passed.
INFO:root:Validation passed.
INFO:root:Validation passed.
INFO:root:Validation passed.
INFO:root:Validation passed.
INFO:root:Validation passed.
INFO:root:Validation passed.
INFO:root:Validation passed.
INFO:root:Validation passed.
INFO:root:Validation passed.
INFO:root:Validation passed.
INFO:root:Validation passed.


Validation of /corpora with metrics included:
Data of corpus Alsatian Drama Corpus passed validation.
Data of corpus Bashkir Drama Corpus passed validation.
Data of corpus Calderón Drama Corpus passed validation.
Data of corpus French Drama Corpus passed validation.
Data of corpus German Drama Corpus passed validation.
Data of corpus German Shakespeare Drama Corpus passed validation.
Data of corpus Greek Drama Corpus passed validation.
Data of corpus Hungarian Drama Corpus passed validation.
Data of corpus Italian Drama Corpus passed validation.
Data of corpus Roman Drama Corpus passed validation.
Data of corpus Russian Drama Corpus passed validation.
Data of corpus Shakespeare Drama Corpus passed validation.
Data of corpus Spanish Drama Corpus passed validation.
Data of corpus Swedish Drama Corpus passed validation.
Data of corpus Tatar Drama Corpus passed validation.


## /corpora/{corpusname} 

In [14]:
from dracor_api_response_schemas import CorpusSchema

In [15]:
corpus_response = get(corpusname="ger", parse_json=True)

In [16]:
#corpus_response

In [17]:
# is_valid(corpus_response, CorpusSchema, catch_error=False)
is_valid(corpus_response, CorpusSchema)

INFO:root:Validation passed.


True

In [18]:
#Validate all
corpusnames = []
for item in corpora_response_no_metrics:
    corpusnames.append(item["name"])

In [19]:
corpusnames

['als',
 'bash',
 'cal',
 'fre',
 'ger',
 'gersh',
 'greek',
 'hun',
 'ita',
 'rom',
 'rus',
 'shake',
 'span',
 'swe',
 'tat']

for corpusname in corpusnames:
    corpus_response = get(corpusname=corpusname, parse_json=True)
    print(f"Validating {corpusname}.")
    is_valid(corpus_response, CorpusSchema, catch_error=False)

## /corpora/{corpusname}/metadata

In [20]:
from dracor_api_response_schemas import PlayMetadataSchema

In [21]:
# metadata_plays_of_corpus_response = get(method="metadata",corpusname="ger",parse_json=True)

In [22]:
# metadata_plays_of_corpus_response[0]

In [23]:
#is_valid(metadata_plays_of_corpus_response[0], PlayMetadataSchema, catch_error=False)

for play in metadata_plays_of_corpus_response:
    is_valid(play, PlayMetadataSchema, catch_error=True)  

In [24]:
# Validate all
for corpusname in corpusnames:
    print(f"Validating corpus {corpusname}.")
    plays_metadata = get(method="metadata",corpusname=corpusname,parse_json=True)
    
    for play in plays_metadata:
        print(f"Validating {play['id']}.")
        is_valid(play, PlayMetadataSchema, catch_error=False)  
    

Validating corpus als.
Validating als000001.
Validating als000008.
Validating als000007.
Validating als000009.
Validating als000010.
Validating als000011.
Validating als000012.
Validating als000006.
Validating als000013.
Validating als000026.
Validating als000014.
Validating als000015.
Validating als000005.
Validating als000027.
Validating als000004.
Validating als000016.
Validating als000017.
Validating als000018.
Validating als000019.
Validating als000028.
Validating als000020.
Validating als000022.
Validating als000021.
Validating als000023.
Validating als000024.
Validating als000003.
Validating als000002.
Validating als000029.
Validating als000025.
Validating als000030.
Validating corpus bash.
Validating bash00001.
Validating bash00002.
Validating bash00003.
Validating corpus cal.
Validating cal000001.
Validating cal000002.
Validating cal000055.
Validating cal000003.
Validating cal000056.
Validating cal000057.
Validating cal000058.
Validating cal000152.
Validating cal000059.
Valida

Validating fre000252.
Validating fre000253.
Validating fre000254.
Validating fre000255.
Validating fre000256.
Validating fre000257.
Validating fre001531.
Validating fre001535.
Validating fre001537.
Validating fre001538.
Validating fre001540.
Validating fre001533.
Validating fre001534.
Validating fre001539.
Validating fre001536.
Validating fre000258.
Validating fre000259.
Validating fre000260.
Validating fre000261.
Validating fre000262.
Validating fre000263.
Validating fre000264.
Validating fre000265.
Validating fre000266.
Validating fre000267.
Validating fre000268.
Validating fre000269.
Validating fre000270.
Validating fre000271.
Validating fre000272.
Validating fre000273.
Validating fre000274.
Validating fre000275.
Validating fre000276.
Validating fre000277.
Validating fre000278.
Validating fre000279.
Validating fre000280.
Validating fre000281.
Validating fre000282.
Validating fre000283.
Validating fre000284.
Validating fre000285.
Validating fre000286.
Validating fre000287.
Validating

Validating fre000741.
Validating fre000742.
Validating fre000743.
Validating fre000744.
Validating fre000745.
Validating fre000746.
Validating fre000747.
Validating fre000748.
Validating fre000749.
Validating fre000750.
Validating fre000751.
Validating fre000752.
Validating fre000753.
Validating fre000754.
Validating fre000755.
Validating fre000756.
Validating fre000757.
Validating fre000758.
Validating fre001499.
Validating fre000759.
Validating fre001500.
Validating fre000760.
Validating fre000761.
Validating fre001546.
Validating fre000762.
Validating fre000763.
Validating fre000764.
Validating fre000765.
Validating fre000766.
Validating fre000767.
Validating fre000768.
Validating fre000769.
Validating fre000770.
Validating fre000771.
Validating fre000772.
Validating fre001547.
Validating fre000773.
Validating fre001548.
Validating fre001550.
Validating fre001549.
Validating fre000774.
Validating fre000775.
Validating fre000776.
Validating fre000777.
Validating fre000778.
Validating

Validating fre001199.
Validating fre001467.
Validating fre001460.
Validating fre001201.
Validating fre001202.
Validating fre001203.
Validating fre001204.
Validating fre001205.
Validating fre001206.
Validating fre001207.
Validating fre001208.
Validating fre001209.
Validating fre001210.
Validating fre001211.
Validating fre001212.
Validating fre001213.
Validating fre001214.
Validating fre001216.
Validating fre001215.
Validating fre001217.
Validating fre001218.
Validating fre001219.
Validating fre001220.
Validating fre001221.
Validating fre001222.
Validating fre001223.
Validating fre001224.
Validating fre001225.
Validating fre001226.
Validating fre001227.
Validating fre001228.
Validating fre001229.
Validating fre001230.
Validating fre001232.
Validating fre001231.
Validating fre001233.
Validating fre001234.
Validating fre001235.
Validating fre001236.
Validating fre001237.
Validating fre001238.
Validating fre001239.
Validating fre001240.
Validating fre001241.
Validating fre001242.
Validating

Validating ger000566.
Validating ger000596.
Validating ger000545.
Validating ger000093.
Validating ger000021.
Validating ger000403.
Validating ger000180.
Validating ger000543.
Validating ger000405.
Validating ger000070.
Validating ger000533.
Validating ger000437.
Validating ger000208.
Validating ger000363.
Validating ger000049.
Validating ger000078.
Validating ger000152.
Validating ger000362.
Validating ger000330.
Validating ger000577.
Validating ger000110.
Validating ger000497.
Validating ger000103.
Validating ger000190.
Validating ger000498.
Validating ger000443.
Validating ger000053.
Validating ger000013.
Validating ger000345.
Validating ger000290.
Validating ger000520.
Validating ger000346.
Validating ger000574.
Validating ger000583.
Validating ger000487.
Validating ger000188.
Validating ger000485.
Validating ger000579.
Validating ger000578.
Validating ger000494.
Validating ger000281.
Validating ger000225.
Validating ger000057.
Validating ger000490.
Validating ger000488.
Validating

Validating greek000026.
Validating greek000004.
Validating greek000008.
Validating greek000022.
Validating greek000011.
Validating greek000005.
Validating greek000020.
Validating greek000032.
Validating greek000012.
Validating greek000035.
Validating greek000027.
Validating greek000017.
Validating greek000010.
Validating greek000016.
Validating greek000033.
Validating greek000037.
Validating greek000009.
Validating greek000014.
Validating greek000001.
Validating greek000039.
Validating greek000003.
Validating greek000029.
Validating greek000013.
Validating greek000034.
Validating greek000028.
Validating greek000002.
Validating greek000023.
Validating greek000030.
Validating greek000031.
Validating greek000006.
Validating greek000019.
Validating greek000040.
Validating greek000007.
Validating greek000025.
Validating greek000021.
Validating greek000018.
Validating greek000036.
Validating greek000015.
Validating greek000038.
Validating greek000024.
Validating corpus hun.
Validating hun000

Validating shake000008.
Validating shake000012.
Validating shake000035.
Validating shake000010.
Validating shake000026.
Validating shake000036.
Validating shake000032.
Validating shake000017.
Validating shake000018.
Validating shake000019.
Validating shake000020.
Validating shake000021.
Validating shake000022.
Validating shake000024.
Validating shake000030.
Validating shake000015.
Validating shake000033.
Validating shake000007.
Validating shake000031.
Validating shake000004.
Validating shake000006.
Validating shake000034.
Validating shake000037.
Validating shake000016.
Validating shake000023.
Validating shake000028.
Validating shake000005.
Validating shake000009.
Validating shake000003.
Validating shake000011.
Validating shake000001.
Validating shake000014.
Validating shake000029.
Validating shake000027.
Validating shake000025.
Validating shake000013.
Validating shake000002.
Validating corpus span.
Validating span000018.
Validating span000001.
Validating span000003.
Validating span0000

OK, all valid...